In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.io as pio
import kaleido 
print('kaleido version:', kaleido.__version__)
from pathlib import Path
import pandas as pd
import numpy as np
import pyarrow
import regex

#Inline figures
pio.renderers.default = 'iframe'

#Themeing
# mcolors = px.colors.qualitative.Dark24

pio.templates['custom'] = go.layout.Template(
    layout=dict(
        xaxis=dict(ticks='outside', tickcolor='lightgray', showgrid=False, showline=True),
        yaxis=dict(ticks='outside', tickcolor='lightgray', showgrid=False, showline=True, mirror=True),
        yaxis2=dict(ticks='outside', tickcolor='lightgray', showgrid=False,),
        # colorway=mcolors,
    )

)

pio.templates.default = 'plotly_white+custom'

kaleido version: 0.2.1


# Transaction Preprocessing

## E-transfer Transaction Features
- `e_at_risk`
    - General indicator
    - Probability that a message references a species at risk, or parts thereof
    - *could also be a binary flag*
    - *can look at CITES for this list*
- `e_cad_at_risk`
    - Export indicator
    - Probability that message references a species at risk in Canada, or parts thereof (bear, geese, etc.)
    - *could also be a binary flag*
    - *get animal list from fintrac doc and papers*
    - Maaz: I don't think e_at_risk as defined above is useful. CITES list is very long. I will just focus on the Canadian at-risk. And I will call it e_at_risk.
        - VVCP12477508, BMIY44394974, ZGAN21196261, MXBU42708741, VYOC17525287, XGAL12621643 -- for sure
        - IBCA52577297 -- 'moose meat'. David Wilkerson sent 978 to Steven Cruz. Wilkerson is also the guy who bought the bear fangs and skin. idk what to write for this.
- `e_role`
    - General indicator
    - Probability that a message references an IWT related role (e.g. poacher, coordinator, supplier, breeder, trader)
    - *could also be a binary flag*
    - Maaz: regex search for these words did not yield anything, nor did semantic search for poacher, nor semantic search for breeder
- `e_trad_med`
    - Import indicator
    - Probability that a message references traditional medicine
    - *could also be a binary flag*
    - Maaz: semantic search for traditional medicine only brought up one possible hit: lion's mane. But, this is a plant. So idk.
 

*Do we want to separate these into TO and FROM?*

In [30]:
DATAPATH = Path('../data/')
EMTPATH = DATAPATH / 'processed' / 'emt.parquet'
emt_df = pd.read_parquet(EMTPATH)

emt_df['trxn_message'] = emt_df['trxn_message'].fillna('')

In [45]:
# function takes iterable of messages and returns list of if its about animal trafficking or no
def classify_message_animal_trafficking(messages):
    messages = pd.Series(messages) # to use str contains

    keywords = [
        "bears?", 
        "cougars?", 
        "goose", 
        "geese", 
        "lynx", 
        "moose", 
        "crab?", 
        "eel", 
        "elver", 
        "lobsters?", 
        "turtles?", 
        "sharks?", 
        "wolf", 
        "wolves",
        "ivory", 
        "tusks?", 
        "scales?", 
        "fins?", 
        "tortoise", 
        "gecko", 
        "salamander", 
        "pangolin",
        "musk deer",
        "macaque",
        "gall",
        "bile",
        "gall ?stones?",
        "tiger",
        "leopard",
        "bones?",
        "rhinoceros",
        "rhino",
        "shell",
        "seahorses?",
        "alligator",
        "crocodile",
        "horns?",
        "skin",
        "fangs?"
    ]
    pattern = '|'.join(['\\b{}\\b'.format(k) for k in keywords])

    negatives = [
        'shark week',
        'shark ?skin',
        'shark tank',
        'lone wolf',
        'bear grylls',
        'fortnite',
        'fornite',
        'snake ?skin',
        'drank ur bear',
        'wolf-brand',
        'bear market inves',
        'wolf of wall street',
        'electric eel',
        'goose island',
        'shark vacuum',
        'lobster dinner',
        'lobster fest',
        'crab shack',
        'bulldog skin care',
        'turtle wax',
        'goose down',
        'phone skin',
        'fitness club',
        'oil change',
        'duck duck goose',
        'kayak',
        'king crab',
        'soap',
        'phone case',
        'pizzeria',
        'kettle',
        'steakhouse',
        'leopard print',
        'art',
        'tiger balm',
        'leopard spot',
        'tiger woods',
        'tiger king',
        'screen protector',
        'donation',
        'seahorse earrings',
        'tickets',
        'power bank',
        'beach towel',
        'diving service',
        'bedliner',
        'camera'
    ]
    negative_pattern = '|'.join(['{}'.format(k) for k in negatives])

    classification = (messages.str.contains(pattern, case=False, na=False)) & (~messages.str.contains(negative_pattern, case=False, na=False))
    return classification.astype(int)

In [46]:
# fuzzy matching with regex example
# regexpr = regex.compile(r"(\bbear\b){e<=1}|(\bfox\b){e<=1}")
# emt_df['e_at_risk'] = emt_df['trxn_message'].map(lambda m : bool(regexpr.search(m)))

# if you want to implement this in the function I wrote, just change how `pattern` is constructed
# pattern = regex.compile('|'.join(['(\\b{}\\b){{e<=1}}'.format(k) for k in keywords]))

# one challenge is e.g. fox would match to for, which gets you lots of false positives
# read https://pypi.org/project/regex/ and you can see that you can specify which character sets to allow for the fuzzy match
# e.g., {s<=2:[a-z]} at most 2 substitutions, which must be in the character set [a-z]
# so to fuzzy match fox but not for, use something like that to exclude matching the `r` character?

# and of course would have to change the str.contains to a map and lambda function

In [47]:
emt_df['e_at_risk'] = classify_message_animal_trafficking(emt_df['trxn_message'])

In [48]:
emt_df[emt_df['e_at_risk']]

KeyError: "None of [Index([0, 0, 0, 0, 0, 0, 0, 0, 0, 0,\n       ...\n       0, 0, 0, 0, 0, 0, 0, 0, 0, 0],\n      dtype='int32', length=506451)] are in the [columns]"

In [ ]:
# function takes iterable of messages and returns list of if its about a traditional medicine plant or not
# I have left this here but I am not sure if this is useful. the animals are already covered in the previous function.
# based on: https://www.canada.ca/en/environment-climate-change/services/convention-international-trade-endangered-species/publications/animal-plant-ingredients-traditional-medicine.html#_03
def classify_message_trad_medicine_plant(messages):
    messages = pd.Series(messages) # to use str contains

    keywords = [
        "aucklandia",
        "costus",
        "root",
        "eagle ?wood",
        "fern",
        "golden haired dog",
        "golden hair dog",
        "nard"
    ]
    pattern = '|'.join(['\\b{}\\b'.format(k) for k in keywords])

    negatives = []
    negative_pattern = '|'.join(['{}'.format(k) for k in negatives])

    classification = (messages.str.contains(pattern, case=False, na=False)) & (~messages.str.contains(negative_pattern, case=False, na=False))

    return classification

In [ ]:
# dummy
def classify_messsage_iwt_role(messages):
    messages = pd.Series(messages)

    classification = [0 for m in messages]

    return classification

In [49]:
emt_df['e_role'] = classify_messsage_iwt_role(emt_df['trxn_message'])

In [50]:
# dummy
def classify_messsage_trad_med(messages):
    messages = pd.Series(messages)

    classification = [0 for m in messages]

    return classification

In [51]:
emt_df['e_trad_med'] = classify_messsage_trad_med(emt_df['trxn_message'])

In [52]:
emt_df.to_parquet(EMTPATH, index=False)
emt_df.sample(3)

,cust_id_sender,cust_id_receiver,name_sender,name_receiver,trxn_message,emt_value,trxn_id,trxn_type,gender_sender,occupation_sender,...,occ_int_receiver,t_to_animal,t_from_animal,t_to_animal_large,t_from_animal_large,t_to_int,t_from_int,e_at_risk,e_role,e_trad_med
257555,CUST20355936,CUST27031313,MATHIEU SIROIS,TERRY SHANNON,,144.0,XBZY34523761,emt,male,Retail Salesperson,...,0.0,0,0,0,0,0,0,0,0,0
265084,CUST22807147,EXTERNAL691911,KENNETH HENDERSON,NOËL CLOUTIER,,162.5,FGPX16617505,emt,male,Real Estate Agent,...,NaN,0,0,0,0,0,1,0,0,0
413302,CUST65918881,CUST87315316,WILLIAM DICKSON,ÉDITH PLANTE,,3.5,ODYG50087098,emt,male,Butcher,...,1.0,0,1,0,0,1,0,0,0,0


## Wire Transfer Transaction Features
- `w_to_country`
    - Import indicator
    - Binary flag
    - 1 if the wire transfer is to a jurisdiction of concern:
        - China
        - Hong Kong
        - South Africa
        - Australia
        - ...
- `w_from_country`
    - *not listed in fintrac doc, but could be a useful flag as well*
    - Binary flag
    - 1 if from a jurisdiction of concern, 0 otherwise
- `w_external_to_animal`
    - Binary flag
    - 1 if from a jurisdiction of concern to an animal related business, 0 otherwise

In [27]:
WIREPATH = DATAPATH / 'processed' / 'wire.parquet'
KYCPATH = DATAPATH / 'processed' / 'kyc.parquet'
EMTPATH = DATAPATH / 'processed' / 'emt.parquet'


SUSCOUNTRIES = ['CN', 'AU']

In [28]:
#Load Data
kyc_df = pd.read_parquet(KYCPATH)
kyc_df = kyc_df[['cust_id', 'occ_animal']]

wire_df = pd.read_parquet(WIREPATH)

#w_to_country
wire_df['w_to_country'] = wire_df['country_receiver'].map(lambda x: 1 if x in SUSCOUNTRIES else 0)

#w_from_country
wire_df['w_from_country'] = wire_df['country_sender'].map(lambda x: 1 if x in SUSCOUNTRIES else 0)

#w_external_to_animal
wire_df['w_external_to_animal'] = wire_df.apply(lambda r: 1 if r.country_sender!='CA' and r.occ_animal_receiver==1 else 0, axis=1)

In [29]:
wire_df.to_parquet(WIREPATH, index=False)
wire_df.sample(3)

,cust_id_sender,cust_id_receiver,name_sender,name_receiver,trxn_value,country_sender,country_receiver,trxn_id,trxn_type,gender_sender,...,occ_int_receiver,t_to_animal,t_from_animal,t_to_animal_large,t_from_animal_large,t_to_int,t_from_int,w_to_country,w_from_country,w_external_to_animal
37526,EXTERNAL258091,CUST72029720,WANG GUI XIANG,PAMELA ROBINSON,2102.0,CA,CA,OWNN52393932,wire,None,...,0.0,0,0,0,0,0,0,0,0,0
60483,CUST35190860,CUST93391146,DR.LISA COLLIER,EHSAAN RATTAN,1002.0,CA,CA,ZSPX98150585,wire,female,...,0.0,0,0,0,0,0,1,0,0,0
23493,CUST74970331,CUST70364516,ERIKA DUNN,REBECCA STEELE,1166.0,CA,CA,EIGO14108342,wire,female,...,0.0,0,0,0,0,0,0,0,0,0


## Cash Transaction Features
- `c_large`
    - General indicator
    - Binary flag
    - 1 if a deposit or withdrawal is an outlier (above Q3+1.5*IQR)
    - To determine outliers, data is grouped by occupation and by transaction type (deposit/withdrawal)

In [17]:
from pathlib import Path
import pandas as pd
import numpy as np
import pyarrow

DATAPATH = Path('../data/')
CASHPATH = DATAPATH / 'processed' / 'cash.parquet'
KYCPATH = DATAPATH / 'processed' / 'kyc.parquet'

UPPERBOUND = 1.5 #Q3 + UPPERBOUND*IQR gives the transaction amount beyond which transactions are classified as outliers

In [20]:
#Load Data
kyc_df = pd.read_parquet(KYCPATH)
kyc_df = kyc_df[['cust_id', 'occupation']]

cash_df = pd.read_parquet(CASHPATH)
cash_df = cash_df.merge(kyc_df, on='cust_id', how='left')

#Get IQR for Outlier Test
q3 = cash_df.groupby(['type', 'occupation'])['trxn_amount'].quantile(0.75)
q1 = cash_df.groupby(['type', 'occupation'])['trxn_amount'].quantile(0.25)
iqr = q3-q1
fence = q3+UPPERBOUND*iqr #upper bound for outlier tests
fence_dict = fence.to_dict()

#Test outliers
def outlier_test(row, fence): 
    outlier = row.trxn_amount > fence[(row.type, row.occupation)]

    return 1 if outlier else 0

cash_df['c_large'] = cash_df.apply(lambda row: outlier_test(row, fence_dict), axis=1)
print(f'Classified {cash_df.c_large.sum()} of {cash_df.c_large.count()} as outliers')

Classified 9774 of 212532 as outliers


In [21]:
cash_df.to_parquet(CASHPATH, index=False)
cash_df.sample(3)

,cust_id,trxn_amount,type,trxn_id,occ_wealth,occ_animal,occ_int,label,occupation,c_large
85037,CUST10799271,1220,deposit,HCLD98205096,0,0,0,0,Business Owner,0
130415,CUST84365555,1040,withdrawal,ROHR59284989,0,0,0,0,Landlord,0
162125,CUST15505402,9060,withdrawal,ILHA85247454,1,0,1,1,Import/Export Business Owner,0


# General Transaction Features
- `t_to_animal`
    - Import indicator
    - Binary flag
    - 1 if the transaction is to an animal related business
- `t_from_animal`
    - *not listed in fintrac doc, but could be a useful flag as well*
    - Binary flag
    - 1 if the transaction is from an animal related business
- `t_to_animal_large`
    - Import indicator
    - Binary flag
    - 1 if the transaction is from an animal related and is above a certain dollar amount, 0 otherwise
    - *tbd the dollar amount*
- `t_from_animal_large`
    - *not listed in fintrac doc, but could be a useful flag as well*
    - Binary flag
    - 1 if the transaction is from an animal related and is above a certain dollar amount, 0 otherwise
    - *tbd the dollar amount*
- `t_to_shipping`
    - Export indicator
    - Binary flag
    - 1 if the transaction is to someone in shipping/postal/cargo
    - *no label for this type of occupation yet*


In [23]:
emt_data = pd.read_csv('../data/raw/emt.csv')
wire_data = pd.read_csv('../data/raw/wire.csv')
cash_data = pd.read_csv('../data/raw/cash.csv')
kyc_data = pd.read_parquet('../data/processed/kyc.parquet')

threshold = 1000

In [24]:
# Add a new column to each dataframe to indicate the type of transaction
cash_data['trxn_type'] = 'cash'
emt_data['trxn_type'] = 'emt'
wire_data['trxn_type'] = 'wire'

cols = ['gender', 'occupation', 'age', 'tenure', 'cust_id', 'occ_wealth', 'occ_animal', 'occ_int']

# Add kyc data to each dataframe
emt_data = pd.merge(emt_data, kyc_data[cols].add_suffix('_sender'), left_on='cust_id_sender', right_on='cust_id_sender', how='left')
emt_data = pd.merge(emt_data, kyc_data[cols].add_suffix('_receiver'), left_on='cust_id_receiver', right_on='cust_id_receiver', how='left')
wire_data = pd.merge(wire_data, kyc_data[cols].add_suffix('_sender'), left_on='cust_id_sender', right_on='cust_id_sender', how='left')
wire_data = pd.merge(wire_data, kyc_data[cols].add_suffix('_receiver'), left_on='cust_id_receiver', right_on='cust_id_receiver', how='left')
cash_data = pd.merge(cash_data, kyc_data[cols + ['name']], on='cust_id', how='left')

emt_data['t_to_animal'] = np.where((emt_data['occ_animal_receiver']==1), 1, 0)
wire_data['t_to_animal'] = np.where((wire_data['occ_animal_receiver']==1), 1, 0)
cash_data['t_to_animal'] = np.where(cash_data['occ_animal']==1, 1, 0)

emt_data['t_from_animal'] = np.where((emt_data['occ_animal_sender']==1), 1, 0)
wire_data['t_from_animal'] = np.where((wire_data['occ_animal_sender']==1), 1, 0)
cash_data['t_from_animal'] = 0

emt_data['t_to_animal_large'] = np.where((emt_data['occ_animal_receiver']==1)  & (emt_data['emt_value'] > threshold), 1, 0)
wire_data['t_to_animal_large'] = np.where((wire_data['occ_animal_receiver']==1)  & (wire_data['trxn_value'] > threshold), 1, 0)
cash_data['t_to_animal_large'] = np.where((cash_data['occ_animal']==1) & (cash_data['trxn_amount'] > threshold), 1, 0)

emt_data['t_from_animal_large'] = np.where((emt_data['occ_animal_sender']==1) & (emt_data['emt_value'] > threshold), 1, 0)
wire_data['t_from_animal_large'] = np.where((wire_data['occ_animal_sender']==1) & (wire_data['trxn_value'] > threshold), 1, 0)
cash_data['t_from_animal_large'] = 0

emt_data['t_to_int'] = np.where((emt_data['occ_int_receiver']==1), 1, 0)
wire_data['t_to_int'] = np.where((wire_data['occ_int_receiver']==1), 1, 0)
cash_data['t_to_int'] = np.where(cash_data['occ_int']==1, 1, 0)

emt_data['t_from_int'] = np.where((emt_data['occ_int_sender']==1), 1, 0)
wire_data['t_from_int'] = np.where((wire_data['occ_int_sender']==1), 1, 0)
cash_data['t_from_int'] = 0

In [26]:
emt_data.to_parquet('../data/processed/emt.parquet')
wire_data.to_parquet('../data/processed/wire.parquet')
cash_data.to_parquet('../data/processed/cash.parquet')